In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import io

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("--- 1. Загрузка данных ---")
try:
    df = pd.read_csv('used_cars.csv')
    print("✅ Файл 'used_cars.csv' успешно прочитан.")
except FileNotFoundError:
    print("❌ Файл не найден. Пожалуйста, загрузите его:")
    uploaded = files.upload()
    df = pd.read_csv(io.BytesIO(uploaded['used_cars.csv']))
    print(f"✅ Файл загружен. Размер: {df.shape}")

print("\n--- Первичный осмотр ---")
print(df.info())
print("\nПервые 5 строк:")
display(df.head())

--- 1. Загрузка данных ---
✅ Файл 'used_cars.csv' успешно прочитан.

--- Первичный осмотр ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB
None

Первые 5 строк:


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [ ]:
print("--- 2. Очистка и Преобразование Типов ---")
df_cleaned = df.copy()

df_cleaned['milage'] = (
    df_cleaned['milage'].astype(str)
    .str.replace(r'\s.*', '', regex=True)
    .str.replace(',', '', regex=False)
    .astype(float)
)

df_cleaned['price'] = (
    df_cleaned['price'].astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

df_cleaned['engine_volume'] = df_cleaned['engine'].astype(str).str.extract(r'(\d+\.?\d*)L').astype(float)

print("\n--- 2.1 Пропуски и Дубликаты ---")
duplicate_count = df_cleaned.duplicated().sum()
df_cleaned = df_cleaned.drop_duplicates().copy()
print(f"Удалено полных дубликатов: {duplicate_count}. Размер после очистки: {df_cleaned.shape}")

missing_values = df_cleaned.isnull().sum()
missing_percent = (missing_values / len(df_cleaned)) * 100
missing_df_summary = pd.DataFrame({'Пропуски (%)': missing_percent.round(2)})
missing_df_summary = missing_df_summary[missing_df_summary['Пропуски (%)'] > 0].sort_values(by='Пропуски (%)', ascending=False)
print("\nПроцент пропусков в столбцах:")
display(missing_df_summary)

df_cleaned.to_csv('used_cars_cleaned.csv', index=False)

--- 2. Очистка и Преобразование Типов ---

--- 2.1 Пропуски и Дубликаты ---
Удалено полных дубликатов: 0. Размер после очистки: (4009, 13)

Процент пропусков в столбцах:


,Пропуски (%)
clean_title,14.87
engine_volume,9.40
fuel_type,4.24
accident,2.82


In [ ]:
print("--- 3. Корреляционный анализ ---")

numerical_cols = ['model_year', 'milage', 'price', 'engine_volume']

df_corr = df_cleaned[numerical_cols].fillna(df_cleaned[numerical_cols].median())

correlation_matrix = df_corr.corr()

print("\nКорреляционная матрица (DataFrame):")
display(correlation_matrix.round(3))

plt.figure(figsize=(9, 7))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Тепловая карта корреляционной матрицы ')
plt.show()

print("\nСильные корреляции с ценой (price, |r| > 0.15):")
price_corr = correlation_matrix['price'].abs().sort_values(ascending=False)
strong_corr = price_corr[price_corr >= 0.15].drop('price', errors='ignore')
display(strong_corr.to_frame(name='Корреляция'))

In [ ]:
print("--- 4. Визуализация признаков ---")

plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['price'][df_cleaned['price'] < 100000].dropna(), kde=True, bins=50, color='skyblue')
plt.title('1. Распределение цен на автомобили (Price) ')
plt.xlabel('Цена (USD)')
plt.ylabel('Количество')
plt.show()

plt.figure(figsize=(10, 6))
df_plot = df_cleaned[df_cleaned['price'] < 100000].dropna(subset=['milage', 'price'])
sns.scatterplot(x='milage', y='price', data=df_plot, alpha=0.6, color='darkgreen')
plt.title('2. Зависимость Цены от Пробега (milage vs price) ')
plt.xlabel('Пробег')
plt.ylabel('Цена (USD)')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='transmission', y='price', data=df_plot[df_plot['price'] < 60000])
plt.title('3. Распределение цен по типу трансмиссии (Transmission) ')
plt.xlabel('Тип трансмиссии')
plt.ylabel('Цена (USD)')
plt.show()

top_brands_price = df_cleaned.groupby('brand')['price'].mean().nlargest(5).reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='brand', y='price', data=top_brands_price, palette='magma')
plt.title('4. Средняя цена ТОП-5 самых дорогих производителей ')
plt.xlabel('Производитель (Brand)')
plt.ylabel('Средняя Цена (USD)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

pivot_table = pd.crosstab(df_cleaned['transmission'], df_cleaned['accident'], normalize='index') * 100
pivot_table.plot(kind='bar', stacked=True, figsize=(8, 6), color=['darkblue', 'orange'])
plt.title('5. Процентное распределение Аварий по типу Трансмиссии ')
plt.xlabel('Тип трансмиссии')
plt.ylabel('Процент автомобилей')
plt.xticks(rotation=0)
plt.legend(title='Была ли авария')
plt.show()

In [ ]:
# --- Код для вывода ключевых метрик в отчете ---
import pandas as pd
import numpy as np

# Загружаем очищенный DataFrame, если он не в памяти
try:
    df_cleaned = pd.read_csv('used_cars_cleaned.csv')
except:
    # Если файл не найден, предполагаем, что df_cleaned определен из Шага 2
    pass

# Расчет корреляции
numerical_cols = ['model_year', 'milage', 'price', 'engine_volume']
df_corr = df_cleaned[numerical_cols].fillna(df_cleaned[numerical_cols].median())
correlation_matrix = df_corr.corr()

print("-------------------------------------------------")
print("КЛЮЧЕВЫЕ ПОКАЗАТЕЛИ ПОСЛЕ EDA")
print("-------------------------------------------------")
print(f"Общее количество строк после очистки: {len(df_cleaned)}")
print(f"Наибольший процент пропусков в столбце 'accident': {df_cleaned['accident'].isnull().mean() * 100:.2f}%")
print(f"Корреляция price с milage (Пробег): {correlation_matrix.loc['price', 'milage']:.3f}")
print(f"Корреляция price с model_year (Год): {correlation_matrix.loc['price', 'model_year']:.3f}")

# Медианная цена по типу трансмиссии
median_price_transmission = df_cleaned.groupby('transmission')['price'].median().round(2)
print("\nМедианная цена по трансмиссии (USD):")
display(median_price_transmission.to_frame(name='Медианная Цена'))

# 📋 Отчет о разведочном анализе данных (EDA)

## Набор данных: Прогнозирование цен на подержанные автомобили

| Характеристика | Значение |
| :--- | :--- |
| **Цель проекта** | Выявить ключевые факторы, влияющие на цену автомобилей. |
| **Размер данных** | ~4000 строк, 12 столбцов. |
| **Ключевая переменная** | `price` (Цена) |

---

## 1. Оценка и Подготовка Данных (Шаг 2)

### 1.1. Качество данных
1. **Преобразование типов:** Успешно преобразованы ключевые признаки (`milage`, `price`, `engine`) из строкового типа (`object`) в **числовой** (`float`) для корректного анализа.
2. **Пропуски:** Обнаружены пропуски в категориальных столбцах (`fuel_type`, `accident`), требующие импутации (заполнения) перед моделированием.
3. **Дубликаты:** Полные дубликаты отсутствуют (0 строк).

---

## 2. Корреляционный анализ признаков (Шаг 3)

Корреляционный анализ выявил линейные взаимосвязи между числовыми признаками.

### 2.1. Матрица Корреляции с `price`

| Признак | Коэффициент Пирсона с `price` | Тип связи |
| :--- | :--- | :--- |
| `milage` (Пробег) | **-0.306** | Умеренная отрицательная |
| `model_year` (Год выпуска) | $0.199$ | Слабая положительная |
| `engine_volume` (Объём) | $0.174$ | Слабая положительная |

* **Вывод:** Главным линейным фактором является **Пробег**. Связь между Годом выпуска и ценой (0.199) является слабой, что указывает на то, что **Бренд** и **Модель** (категориальные признаки) играют более важную роль в ценообразовании.

---

## 3. Ключевые Находки по Визуализациям (Шаг 4)

### 3.1. Распределение Цены (`price`)
* **Находка:** Гистограмма показала сильную **правостороннюю асимметрию** (скос вправо) , что типично для финансовых данных. Большинство авто дешевые, но есть "хвост" дорогих выбросов.
* **Необходимость:** Требуется логарифмическое преобразование целевой переменной для нормализации перед обучением регрессионной модели.

### 3.2. Взаимосвязь Цены и Пробега
* **Находка:** Диаграмма рассеивания демонстрирует **нелинейную** отрицательную зависимость , подтверждая, что цена быстро падает на малых пробегах.

### 3.3. Влияние Категориальных Признаков
* **Бренд:** Агрегация показала, что **бренд** является ключевым фактором: средняя цена ТОП-5 люксовых брендов (например, Mercedes) значительно выше, чем у массовых производителей.
* **Трансмиссия:** Ящики с усами показывают, что **Автоматическая** трансмиссия имеет более высокую медианную цену и более высокий ценовой потенциал.

---

## 4. Заключение и Рекомендации для Моделирования

| Цель | Рекомендация | Причина |
| :--- | :--- | :--- |
| **Нормализация** | Применить `np.log(price)` к целевой переменной. | Устранение асимметрии и снижение влияния выбросов. |
| **Категории** | Применить **кодирование** (`One-Hot Encoding` или `Target Encoding`) к `brand`, `model`, `fuel_type`, `transmission`. | Учесть сильное нелинейное влияние этих факторов на цену. |
| **Модель** | Использовать ансамблевые методы (GBM, Random Forest). | Эти модели лучше улавливают **нелинейные** зависимости, выявленные в ходе EDA. |